In [1]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader


/home/victor/Documents/travail_epita/venv_epita/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
url="https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip"
data_path = util.download_and_unzip(url,"./data")
print(data_path)
corpus,queries,qrels = GenericDataLoader(data_folder=data_path).load(split="test")

./data/dbpedia-entity


  0%|          | 0/140724 [00:00<?, ?it/s]

In [3]:
print("size of corpus is",len(corpus))
print("size of queries is", len(queries))
print("size of qrels is", len(qrels))

size of corpus is 140724
size of queries is 400
size of qrels is 400


We have 3 values.
-Corpus is a dictionary representing the description of each DBPedia page, representing an event, a person, a culture of anything else. Each element element contains the title of a page and the text associated.
-Queries represent a series of request by a user. It could be a question, a name, or a sentence describing what he looking for.
-Qrels represent the possible answers that match with queries. Each values is marked between 0 and 2. The best they have the best the response is.

In [4]:
set_usefull_themes = set()
for q in qrels:
    for t in qrels[q]:
        set_usefull_themes.add(t)
set_all_theme = set()

In [5]:
import tqdm
obj = set([el for el in tqdm.tqdm(corpus)])

100%|██████████| 140724/140724 [00:00<00:00, 3227537.98it/s]


In [6]:
import random
number_trash = 100000
obj = obj.difference(set_usefull_themes)
len(obj)


100000

In [7]:
trash_themes = random.sample(obj,k=number_trash)
set_usefull_themes = set_usefull_themes.union(trash_themes)
len(set_usefull_themes)

/tmp/ipykernel_19024/1560637768.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  trash_themes = random.sample(obj,k=number_trash)


140724

In [8]:
len(trash_themes)

100000

In [9]:
corpus2 = corpus.fromkeys(set_usefull_themes)

In [10]:
len(corpus2)

140724

In [11]:
corpus2 = {key:value for (key,value) in corpus.items() if key in set_usefull_themes}
corpus2

{'<dbpedia:ASCII>': {'text': {'text': 'ASCII (/ˈæski/ ASS-kee), abbreviated from American Standard Code for Information Interchange, is a character-encoding scheme (the IANA prefers the name US-ASCII).  ASCII codes represent text in computers, communications equipment, and other devices that use text. Most modern character-encoding schemes are based on ASCII, though they support many additional characters.',
   'title': 'ASCII'},
  'title': 'ASCII'},
 '<dbpedia:Academy_Awards>': {'text': {'text': 'The Academy Awards or The Oscars is an annual American awards ceremony honoring cinematic achievements in the film industry. The various category winners are awarded a copy of a statuette, officially the Academy Award of Merit, which is better known by its nickname Oscar.',
   'title': 'Academy Awards'},
  'title': 'Academy_Awards'},
 '<dbpedia:Apollo>': {'text': {'text': 'Apollo (Attic, Ionic, and Homeric Greek: Ἀπόλλων, Apollōn (GEN Ἀπόλλωνος); Doric: Ἀπέλλων, Apellōn; Arcadocypriot: Ἀπείλω

In [12]:
list_json = []
for el in tqdm.tqdm(corpus2):
    list_json.append({"_id":el,"title":el[9:-1],"text":corpus2[el],"metadata":f"<http://dbpedia.org/resource/{el[9:-1]}>"})


100%|██████████| 140724/140724 [00:00<00:00, 179664.73it/s]


In [13]:
import json
with open("corpus2.jsonl","w") as file:
    for item in tqdm.tqdm(list_json):
        file.write(json.dumps(item) + "\n")


100%|██████████| 140724/140724 [00:00<00:00, 143417.27it/s]
